In [1]:
from tavily import TavilyClient
from typing import Annotated, Sequence, TypedDict,Literal,Dict
from dotenv import load_dotenv  
from langchain_core.messages import BaseMessage # The foundational class for all message types in LangGraph
from langchain_core.messages import ToolMessage # Passes data back to LLM after it calls a tool such as the content and the tool_call_id
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage # Message for providing instructions to the LLM
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END,START,MessagesState
from langgraph.prebuilt import ToolNode,create_react_agent
import os
import requests
from bs4 import BeautifulSoup
from langchain_core.prompts import ChatPromptTemplate
from tools.pull_businesstoday import get_topic_news
from tools.get_current_timestamp import get_current_time
load_dotenv()
from tools.techinacalAnalysis.get_indicators import get_indicator
from tools.techinacalAnalysis.get_momentum_stocks import get_momentum_stocks
from tools.techinacalAnalysis.golden_signal import golden_signal_exit,golden_signal_entry
from tools.fundamentalAnalysis.financeData import get_fundamental_summary
import warnings
warnings.filterwarnings('ignore')



In [2]:
class SupervisorState(MessagesState):
    """State for the multi-agent system"""
    messages: Annotated[Sequence[BaseMessage], add_messages]
    next_agent: str = ""
    news_data: str = ""
    news_analysis: str = ""
    techincal_analysis: str = ""
    fundamental_analysis: str=""
    startegy: str=""
    final_report: str = ""
    task_complete: bool = False
    current_task: str = ""
    sender: str=""
    user_question : str=""

In [3]:
model = ChatOpenAI(model = "gpt-4.1")

In [4]:
def create_supervisor_chain():
    """ Creates the supervisor decision chain"""
    
    
    supervisor_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are the supervisor agent for a multi-agent stock-trading assistant.
Your job is to detect the user’s intent, decide which agent should act next,
and ensure the final output is produced by the stock_recommender.

--------------------------------------------------
STATE AWARENESS (CRITICAL)
--------------------------------------------------

Current state:
- Has technical_report: {has_technical} ,agent:TechnicalMarketSignalAgent
- Has fundamental_report: {has_fundamental}, agent:FundamentalAnalysisAgent
- Has news_analysis: {has_analysis}, agent: FinancialNewsInsightAgent
- Has final_recommendation: {has_report} agent:stock_recommender

IMPORTANT:
- NEVER call an agent if its required output already exists.
- For each intent, call each required agent AT MOST ONCE.
- If all required reports for the intent are present,
  immediately route to stock_recommender.
- If final_recommendation exists, respond with DONE.


--------------------------------------------------
INTENT DETECTION
--------------------------------------------------

First, identify the user intent:

1) SINGLE STOCK ANALYSIS
- User mentions or implies a specific stock/ticker.
- Required data:
  • Technical analysis
  • Fundamental analysis
  • News analysis (to check recent stock-specific news)
- After required reports are available, route to stock_recommender.
- User might need only 1-2 analysis so check that and plan before proceeding 

2) GOOD STOCKS / INVESTMENT IDEAS
- User asks for “good stocks”, “stocks to invest”, “best stocks”, etc.
- Required data:
  • Technical screening (momentum + golden signals)
  • News scan for trending stocks (To see which sector and stock is trending)
- After required data is available, route to stock_recommender.

3) EXPLICIT REQUESTS
- “golden signal” → TechnicalMarketSignalAgent
- “momentum stocks” → TechnicalMarketSignalAgent
- “technical only, technically strong” → TechnicalMarketSignalAgent
- “fundamentals only or fundamentals details etc” → FundamentalAnalysisAgent
- “news only or recent news ” → FinancialNewsInsightAgent
- If recommendation is explicitly requested, route to stock_recommender
  after collecting missing required data.
- don't call any agent if its already called . Check from current state

4) OUT-OF-SCOPE OR GENERAL QUESTIONS
- If the user’s question is NOT related to:
  • stock analysis
  • stock selection
  • market news
  • technical or fundamental analysis
- Then respond politely with a brief clarification of scope.
- Do NOT call any sub-agent.
- Ask the user to rephrase or ask a stock-market-related question.
- always mention which type of request you classified 


--------------------------------------------------
AGENTS
--------------------------------------------------

- FinancialNewsInsightAgent
  → Summarizes relevant financial news with limited sentiment.

- TechnicalMarketSignalAgent
  → Provides momentum stocks, golden signals, or technical data and also helps in technical analysis for single stocks.
  → Always pass valid yfinance tickers or screening instructions.

- FundamentalAnalysisAgent
  → Produces a structured, factual fundamental report.

- stock_recommender (FINAL STEP)
  → Combines available technical, fundamental, and news data
    into a clean, user-ready recommendation.

Always frame an appropriate question for the subagent you are calling 
- Don't call any agent if it's already called. Check from the current state

--------------------------------------------------
RULES
--------------------------------------------------

• Use the minimum number of agent calls.
• Do NOT repeat the same agent for the same intent.
• Explicit requests must not trigger other agents.
• The stock_recommender must be the final step for investment-related queries.
• Out-of-scope questions must be answered politely without agent calls.

--------------------------------------------------
CONTEXT
--------------------------------------------------

Original User Query: {question}
Last Agent Message: {task}

--------------------------------------------------
OUTPUT FORMAT (STRICT)
--------------------------------------------------

Respond with exactly ONE of the following:

TechnicalMarketSignalAgent, <task>
FundamentalAnalysisAgent, <task>
FinancialNewsInsightAgent, <task>
stock_recommender, <task>
DONE
POLITE_OUT_OF_SCOPE_RESPONSE, <short reply to user>


""")
    ])
    
    return supervisor_prompt | model    

In [5]:
def supervisor_agent(state: SupervisorState) -> Dict:
    
    messages = state["messages"]
    task = messages[-1].content if messages else "No task"
    question=state["user_question"]
    
    # Check what's been completed
    has_technical = bool(state.get("techincal_analysis", ""))
    has_analysis = bool(state.get("news_analysis", ""))
    # has_strategy = bool(state.get("startegy", ""))
    # has_salesReport = bool(state.get("sale_report", ""))
    has_report = bool(state.get("final_report", ""))
    has_fundamental=bool(state.get("fundamental_analysis",""))

    
    # Get LLM decision
    chain = create_supervisor_chain()
    decision = chain.invoke({
        "task": task,
        "has_analysis": has_analysis,
        "has_technical":has_technical,
        # "has_strategy": has_strategy,
        # "has_salesReport": has_salesReport,
        "has_report": has_report,
        "question": question,
        "has_fundamental":has_fundamental

    })

    print("Yo yo:----",{
        "task": task,
        "has_analysis": has_analysis,
        "has_technical":has_technical,
        # "has_strategy": has_strategy,
        # "has_salesReport": has_salesReport,
        "has_report": has_report,
        "question": question,
        "has_fundamental":has_fundamental

    })
    
    # Parse decision
    decision_text = decision.content.strip().lower()
    supervisor_msg="Supervisor: "+ decision_text

    print("supervisor_msg:-",supervisor_msg)

    # Determine next agent
    if "done" in decision_text or has_report:
        next_agent = "end"
    elif "financialnewsinsightagent" in decision_text :
        next_agent = "FinancialNewsInsightAgent"
    elif "fundamentalanalysisagent" in decision_text:
        next_agent = "FundamentalAnalysisAgent"
    elif "technicalmarketsignalagent" in decision_text :
        next_agent = "TechnicalMarketSignalAgent"
    elif "stock_recommender" in decision_text :
        next_agent = "stock_recommender"
    else:
        next_agent = "end"
    
    return {
        "messages": [AIMessage(content=supervisor_msg)],
        "next_agent": next_agent,
        "current_task": supervisor_msg
    }

In [6]:
def FundamentalAnalysisAgent(state:SupervisorState)->dict:
    task = state.get("current_task")
    print("----Fundamental_analysizer----")
    prompt=f"""
    You are the FundamentalAnalysisAgent. 
Your job is to take the structured fundamental data provided by the tool 
(get_fundamental_summary) and convert it into a clear, concise, and accurate 
fundamental analysis report for the ticker.

Task assigned by supervisor: {task}

Tool Description:
- get_fundamental_summary: A tool that takes a yfinance-compatible ticker and 
  returns a structured JSON containing the stock’s key fundamental details 
  including valuation metrics, yearly and quarterly financial summaries, 
  and growth indicators.

The tool output may be:
- a structured JSON object, OR
- a JSON-formatted string

If the input is a JSON string, first parse it mentally and then use the data.
Do NOT fail or complain about formatting.

You must:
- Interpret valuation metrics (PE, PB, Market Cap, ROE, Margins, Dividend Yield)
- Summarize yearly and quarterly financial performance
- Highlight revenue and profit trends (growth or weakness)
- Use the CAGR values (if present) to describe long-term growth
- Identify strengths, weaknesses, and notable signals
- Keep the summary factual, short, and useful for investment decision-making
- Do NOT speculate or predict future prices
- Do NOT include any text unrelated to fundamentals (like technical analysis)

--------------------------------------------------
OUTPUT REQUIREMENTS (STRICT)
--------------------------------------------------

Return a structured **fundamental data report only**.

### Fundamental Report

**Valuation Data**
- List valuation metrics exactly as provided:
  PE, PB, Market Cap, ROE, Margins, Dividend Yield, etc.

**Yearly Financial Data**
- Present yearly figures exactly as provided:
  Revenue, Net Income, EPS, EBIT/EBITDA (if available)
- Preserve all numeric values and reporting periods.

**Quarterly Financial Data**
- Present quarterly figures exactly as provided:
  Revenue, Net Income, EPS, EBIT/EBITDA (if available)
- Preserve all numeric values and reporting periods.

**Growth Metrics**
- List revenue CAGR and profit CAGR if present.

**Summary**
- A short factual recap stating what data is included
  (e.g., valuation data, yearly financials, quarterly financials, growth metrics).
- Do NOT add interpretation or commentary.

--------------------------------------------------
FORMATTING GUIDELINES
--------------------------------------------------

- Keep numbers intact.
- Use clear headings and readable formatting.
- Avoid unnecessary text.
- Output must be easy for another LLM to consume for further analysis.


------------------------------------------------------------

"""
    tools=[get_fundamental_summary]
    # toolmodel=model.bind_tools(tools)
    # response = toolmodel.invoke([HumanMessage(content=prompt)])


    reactagent = create_react_agent(
    model=model,
    prompt=prompt,
    tools=tools)
    input = {"messages":[{"role": "user","content":" You are the FundamentalAnalysisAgent."}]}
    response = reactagent.invoke(input)
    response=response["messages"][-1]
    final_response=f"fundamental_analysis: {response.content}"

    

       
    return {
        "messages": [AIMessage(content=final_response)],
        "fundamental_analysis": response.content,
        "next_agent": "supervisor"
    }

In [7]:
def TechnicalMarketSignalAgent(state:SupervisorState)->dict:
    task = state.get("current_task")
    print("----technical_analysizer----")
    prompt=f"""You are the TechnicalMarketSignalAgent in a multi-agent stock trading system.
Your job is to interpret the supervisor’s task, decide whether the request 
is for (1) a list of technically strong or weak stocks, or (2) technical 
analysis of specific tickers, then call the appropriate tools.


Task assigned by supervisor: {task}

------------------------------------------------------------
### Capabilities
------------------------------------------------------------
You can perform two categories of work:

1. **Stock Screening (Lists)**
   Use when the supervisor's task mentions:
   - good stocks / strong stocks / weak stocks
   - momentum stocks
   - bullish or bearish signals
   - golden cross / trend signals
   Tools:
     - get_momentum_stocks
     - get_golden_signal_entry
     - get_golden_signal_exit

2. **Technical Analysis for Specific Tickers**
   Triggered when a task includes or implies a particular ticker.
   Use:
     - get_indicator(ticker, indicator or None)
     ticket should be proper as yfinance library like('RELIANCE.NS', 'KIRLOSBROS.NS')
   You may call it once for all indicators or multiple times if needed.

------------------------------------------------------------
### Tool Selection Logic
------------------------------------------------------------
- If the task is generic (e.g., “give good stocks”), generate stock lists.
- If the task includes a ticker, perform indicator-based analysis.
- If the task requires list → analysis, perform in that sequence.
- Use only the minimum tools required to satisfy the task.
- Never guess indicator values; always rely on tool outputs.

------------------------------------------------------------
### Output Requirements (Donot remove much information)
------------------------------------------------------------
Return a structured **technical report only**.
Do NOT provide analysis, opinions, predictions, or recommendations.

**Technical Data**
- Present raw or summarized outputs from tools:
  - momentum stock lists
  - golden signal entry/exit lists
  - indicator values
- You may briefly state what an indicator represents (factual description only).

**Indicator Summary**
- List indicator values exactly as returned:
  RSI, MACD, EMA, ADX, Volume, Bollinger Bands, etc.
- Format clearly for readability.
- Do NOT add qualitative judgment.

**Signal Summary**
- If screening outputs are present (momentum, golden signals),
  list them cleanly and factually.

**Overall Technical Summary**
- Provide a short factual recap of the technical data returned.
- No interpretation, no comparison, no investment commentary.

The output should be numeric, well-formatted, and easy for another LLM
to consume for further analysis.

------------------------------------------------------------

"""
    tools=[get_indicator,get_momentum_stocks,golden_signal_exit,golden_signal_entry]
    # toolmodel=model.bind_tools(tools)
    # response = toolmodel.invoke([HumanMessage(content=prompt)])


    reactagent = create_react_agent(
    model=model,
    prompt=prompt,
    tools=tools)
    input = {"messages":[{"role": "user","content":"You are the TechnicalMarketSignalAgent in a multi-agent stock trading system.Your job is to analyze stocks using technical indicators and signals by callingthe correct tools and combining their outputs intelligently."}]}
    response = reactagent.invoke(input)
    response=response["messages"][-1]
    final_response=f"techinal_analysis: {response.content}"

    

       
    return {
        "messages": [AIMessage(content=final_response)],
        "techincal_analysis": response.content,
        "next_agent": "supervisor"
    }

In [8]:
def FinancialNewsInsightAgent(state:SupervisorState)->dict:
    task = state.get("current_task")
    print("----news_summary_creator----")


    prompt = f""" You are the FinancialNewsInsightAgent, a unified financial news intelligence agent 
in a multi-agent stock-trading system.

Your responsibilities include:
1. **Summarizing raw financial news** scraped from sources like Moneycontrol, 
   Business Today, Mint, Reuters, etc.
2. **Performing sentiment and market-impact analysis** on the summarized news.

---------------------------------------------
### PART 1 — NEWS SUMMARIZATION (STRICT, FACT-PRESERVING)
---------------------------------------------
You will receive raw web-scraped content that may include clutter.

Your task:
- Extract and summarise **only factual, finance-relevant news**.
- Retain all important details:
  - company names  
  - financial figures  
  - announcements  
  - dates, deals, guidance, earnings, macro events  
  - sector-level details  
- Ignore ONLY UI elements, ads, menus, unrelated text.
- Summaries must be **clean, concise, and fact-preserving**.
- Summaries must include:
  - **Current timestamp at the top**
  - **A short headline**
  - **Bullet-point factual summary**

Do NOT:
- Add sentiment
- Add predictions
- Infer or hallucinate missing details
- Remove important details

---------------------------------------------
### PART 2 — SENTIMENT + MARKET IMPACT ANALYSIS (Donot remove much informations)
---------------------------------------------
Using ONLY the summary you generated above:

For each news item:
- Classify sentiment as: **Positive / Negative / Neutral**
- Identify affected:
  - companies
  - sectors
  - indices (if applicable)
- Provide a short, factual insight on expected market impact.
  Keep it grounded in financial reasoning — no speculation beyond the information.

---------------------------------------------
### OUTPUT FORMAT
---------------------------------------------
Return response in the following format:

[Timestamp]

### Headline
<one-line summary>

### Factual Summary
- bullet point 1
- bullet point 2
- bullet point 3
...

### Sentiment
Positive / Negative / Neutral

### Affected Stocks / Sectors
- list

### Market Impact Insight
<2–3 lines of reasoning>

---------------------------------------------

Your task assigned by supervisor: {task}
"""
    tools=[get_topic_news,get_current_time]
    # toolmodel=model.bind_tools(tools)
    # response = toolmodel.invoke([HumanMessage(content=prompt)])


    reactagent = create_react_agent(
    model=model,
    prompt=prompt,
    tools=tools)
    input = {"messages":[{"role": "user","content":"You are a financial news summarizer working with web-scraped content from known news websites (e.g., Moneycontrol, Business Today)."}]}
    response = reactagent.invoke(input)
    response=response["messages"][-1]
    final_response=f"news_summarizer: here is the summary of the news : {response.content}"

    

       
    return {
        "messages": [AIMessage(content=final_response)],
        "news_analysis": response.content,
        "next_agent": "supervisor"
    }
    
    

In [9]:
def stock_recommender(state:SupervisorState)->dict:
    task = state.get("current_task"," recommend stock with all the info ")
    news_report=state.get("news_analysis","Now news please data")
    technical_analysis=state.get("techincal_analysis","Now data")
    fundamental_analysis=state.get("fundamental_analysis","Now news please data")

    print("---stock_predictor----")


    prompt=f"""
You are the stock_recommender agent.
Your job is to generate a clear, well-structured, investment report using ONLY
the information provided by other agents (technical, fundamental, news).

Some inputs may be missing. Use whatever data is available.
Do NOT guess, infer, or invent any information.

--------------------------------------------------
CORE RESPONSIBILITIES
--------------------------------------------------

1. Identify whether the request is for:
   - a SINGLE stock, or
   - MULTIPLE stocks (selection or ranking).

2. TECHNICAL ANALYSIS USAGE
   - Comment on EACH available indicator (RSI, MACD, EMA, ADX, Volume, Bollinger Bands, etc.).
   - Explain what each indicator value suggests using basic technical rules, for example:
       • RSI above/below key levels
       • MACD signal direction or crossover
       • EMA alignment (short vs long term)
       • ADX trend strength
       • Volume expansion or contraction
   - Combine indicators to give an overall technical view.
   - Do NOT introduce indicators that were not provided.

3. FUNDAMENTAL ANALYSIS USAGE
   - Always include actual numbers when fundamentals are available.
   - Focus on the MOST RECENT 1–2 YEARS of data only.
   - Comment on:
       • Sales / revenue trend
       • Profit or EPS trend
       • Any visible improvement or decline
   - Clearly separate yearly and quarterly observations if both are present.
   - Do NOT assume future growth or valuation changes.

4. NEWS ANALYSIS USAGE
   - Use news only to identify:
       • which stocks are currently in focus or trending
       • positive, negative, or neutral market impact
   - Do NOT exaggerate news impact.
   - Link news relevance clearly to the stock(s) mentioned.

5. FINAL INVESTMENT VIEW
   - Provide a balanced stance: Bullish / Neutral / Cautious.
   - Every stance MUST be supported by:
       • technical reasons
       • fundamental reasons (if available)
       • news impact (if available)
   - If multiple stocks are provided:
       • clearly highlight the stronger candidates
       • explain why they stand out relative to others

--------------------------------------------------
REPORT STYLE GUIDELINES
--------------------------------------------------

- You may choose your own report structure.
- The report should read like a professional investment note.
- Always explain the **reason** behind each conclusion.
- Preserve all important numerical details.
- Avoid unnecessary repetition.
- Do NOT add opinions unsupported by the data.
- Do NOT hallucinate missing information.

--------------------------------------------------
AVAILABLE INPUTS
--------------------------------------------------

Technical Analysis:
{technical_analysis}

Fundamental Analysis:
{fundamental_analysis}

News Analysis:
{news_report}

Task from supervisor:
{task}


--------------------------------------------------
Available Inputs:
------------------------------------------------------------
technical_analysis: {technical_analysis}  
news_analysis: {news_report}  
fundamental_analysis: {fundamental_analysis}  
Task by supervisior: {task}

    
    """

    response = model.invoke([HumanMessage(content=prompt)])
    final_report = response.content

    return {
        "messages": [AIMessage(content=final_report)],
        "final_report": final_report,
        "next_agent": "supervisor"
    }
    
    
    

In [10]:
def router(state: SupervisorState) -> Literal["supervisor", "TechnicalMarketSignalAgent", "FinancialNewsInsightAgent", "stock_recommender","FundamentalAnalysisAgent", "tool_call","__end__"]:


    messages = state["messages"]
    last_message = messages[-1]
    

    
    next_agent = state.get("next_agent", "supervisor")
    print("router:-",next_agent)
    
    if next_agent == "end" or state.get("task_complete", False):
        return "__end__"

    if next_agent in ["supervisor", "TechnicalMarketSignalAgent", "FinancialNewsInsightAgent", "stock_recommender","FundamentalAnalysisAgent"]:
        return next_agent
        
    return "supervisor"

In [11]:
from langgraph.checkpoint.memory import MemorySaver


In [12]:
workflow = StateGraph(SupervisorState)

# Add nodes
workflow.add_node("supervisor", supervisor_agent)
workflow.add_node("TechnicalMarketSignalAgent", TechnicalMarketSignalAgent)
workflow.add_node("FinancialNewsInsightAgent", FinancialNewsInsightAgent)
workflow.add_node("stock_recommender", stock_recommender)
workflow.add_node("FundamentalAnalysisAgent",FundamentalAnalysisAgent)



# Set entry point
workflow.set_entry_point("supervisor")

# Add routing
for node in ["supervisor", "FinancialNewsInsightAgent", "stock_recommender","TechnicalMarketSignalAgent","FundamentalAnalysisAgent"]:
    workflow.add_conditional_edges(
        node,
        router,
        {
            "supervisor": "supervisor",
            "FinancialNewsInsightAgent": "FinancialNewsInsightAgent",
            "stock_recommender": "stock_recommender",
            "TechnicalMarketSignalAgent":"TechnicalMarketSignalAgent",
            "FundamentalAnalysisAgent":"FundamentalAnalysisAgent",
            "__end__": END
        }
    )

# workflow.add_conditional_edges(
#     "call_tool",
#     # Each agent node updates the 'sender' field
#     # the tool calling node does not, meaning
#     # this edge will route back to the original agent
#     # who invoked the tool
#     lambda x: x["sender"],
#     {
#         "news_summarizer": "news_summarizer"
       
#     },
# )

checkpointer = MemorySaver()

graph=workflow.compile()


# config={"configurable":{"thread_id":125}}



In [13]:
 # while(True):
     
 #     user_qury=input()
 #     inputs = {"user_question": [(f"user,{user_qury}")]}


 #     response=graph.invoke(inputs,config=config)
 #     SupervisorState=response
     
 #     for message in response["messages"][-2:]:
 #         message.pretty_print()

In [14]:
prompts="""
How is sbi ? """

inputs = {"user_question": [(f"user,{prompts}")]}


response=graph.invoke(inputs)


Yo yo:---- {'task': 'No task', 'has_analysis': False, 'has_technical': False, 'has_report': False, 'question': ['user,\nHow is sbi ? '], 'has_fundamental': False}
supervisor_msg:- Supervisor: technicalmarketsignalagent, please provide a technical analysis for sbi (ticker: sbin.ns), including recent price trends, support/resistance levels, and any significant technical signals.
router:- TechnicalMarketSignalAgent
----technical_analysizer----


[*********************100%***********************]  1 of 1 completed


router:- supervisor
Yo yo:---- {'task': 'techinal_analysis: Technical Report: SBIN.NS (State Bank of India)\n\nTechnical Data\n\nIndicator Summary\n\n- RSI (Relative Strength Index): 51.46\n  - RSI values typically range between 0-100. It measures the speed and change of price movements; a level around 50 indicates neutral momentum.\n\n- ADX (Average Directional Index): 31.51\n  - ADX indicates the strength of a trend. Readings above 25 suggest a strong trend is present.\n\n- EMA (Exponential Moving Average, 14-period): 969.15\n\n- MACD (Moving Average Convergence Divergence):\n  - MACD Value: 4.78\n  - Signal Value: 6.02\n  - Histogram Value: -1.24\n  - Trend: Bearish\n  - Crossover: None\n  - Momentum: Decreasing\n  - Previous MACD: 5.49\n  - Previous Signal: 6.33\n  - Previous Histogram: -0.84\n\n- Volume:\n  - Last Volume: 3,286,321\n  - 20-day Average Volume: 8,249,137.35\n  - Volume Trend: Low\n\n- Bollinger Bands:\n  - Upper Band: 984.82\n  - Middle Band: 966.77\n  - Lower Band:

In [15]:
for msg in response["messages"]:
    print(msg.pretty_print())

================================== Ai Message ==================================

Supervisor: technicalmarketsignalagent, please provide a technical analysis for sbi (ticker: sbin.ns), including recent price trends, support/resistance levels, and any significant technical signals.
None
================================== Ai Message ==================================

techinal_analysis: Technical Report: SBIN.NS (State Bank of India)

Technical Data

Indicator Summary

- RSI (Relative Strength Index): 51.46
  - RSI values typically range between 0-100. It measures the speed and change of price movements; a level around 50 indicates neutral momentum.

- ADX (Average Directional Index): 31.51
  - ADX indicates the strength of a trend. Readings above 25 suggest a strong trend is present.

- EMA (Exponential Moving Average, 14-period): 969.15

- MACD (Moving Average Convergence Divergence):
  - MACD Value: 4.78
  - Signal Value: 6.02
  - Histogram Value: -1.24
  - Trend: Bearish
  - Crossove